In [ ]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')

s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [ ]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta
from pandas import Timestamp

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase

In [ ]:
#import data with the users mails
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Coverage%20Advisor%20Tracker%20OFICIAL.xlsx"\
    ,filename = 'Coverage Advisor Tracker OFICIAL.xlsx')
ca = pd.read_excel ('Coverage Advisor Tracker OFICIAL.xlsx')
ca

In [14]:


def format_date_column(serie):
    format_date = []
    for index, value in serie.iteritems():
        if type(value) != datetime and type(value) != Timestamp:  
            if type(value) == float:
                value = datetime.strptime('1900-1-1 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
            else: 
                value = datetime.strptime('1950-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
        else:
            value = datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S')
            format_date.append(value)
    
    return format_date
    


In [ ]:
#for a, b in ca['Date Scheduled'].iteritems():
#    if type(b) != float and type(b) != datetime:
#        print(b)
#        print(type(b))

In [ ]:
#ca['PAR Requested'] = format_date_column(ca['PAR Requested'])
#ca['Date First Contact'] = format_date_column(ca['Date First Contact'])
#ca['PAR Received'] = format_date_column(ca['PAR Received'])
#ca['Date Analysis'] = format_date_column(ca['Date Analysis'])
#ca['Date Invite Sent'] = format_date_column(ca['Date Analysis'])
#ca['Date Scheduled'] = format_date_column(ca['Date Scheduled'])

In [ ]:
#Compare column R-W

In [ ]:
equality_relation = ca[['Request ID', 'Sav ID','Coverage Advisor','PAR Requested','Date First Contact','PAR Received','Date Analysis','Date Invite Sent','Date Scheduled']]

In [ ]:
equality_relation['PAR Requested'] = format_date_column(equality_relation['PAR Requested'])
equality_relation['Date First Contact'] = format_date_column(equality_relation['Date First Contact'])
equality_relation['PAR Received'] = format_date_column(equality_relation['PAR Received'])
equality_relation['Date Analysis'] = format_date_column(equality_relation['Date Analysis'])
equality_relation['Date Invite Sent'] = format_date_column(equality_relation['Date Analysis'])
equality_relation['Date Scheduled'] = format_date_column(equality_relation['Date Scheduled'])

In [ ]:
equality_relation

In [ ]:
equality_relation['R_less_S'] = int(0)
equality_relation['S_less_T'] = int(0)
equality_relation['T_less_U'] = int(0)
equality_relation['U_less_V'] = int(0)
equality_relation['V_less_W'] = int(0)

In [ ]:
for i, value in equality_relation.iterrows():
    if value['PAR Requested'] > value['Date First Contact']:
        equality_relation['R_less_S'].iloc[i] = 1

    if value['Date First Contact'] > value['PAR Received']:
       equality_relation['S_less_T'].iloc[i] = 1  

    if value['PAR Received'] > value['Date Analysis']:
        equality_relation['T_less_U'].iloc[i] = 1  

    if value['Date Analysis'] > value['Date Invite Sent']:
        equality_relation['U_less_V'].iloc[i] = 1  

    if value['Date Invite Sent'] > value['Date Scheduled']:
        equality_relation['V_less_W'].iloc[i] = 1  

In [ ]:
equality_relation.to_excel('test.xlsx')